# Preprocessing for the Chest X-ray dataset

Dataset originated from the paper of Wang et al. 2016 and extracted from the official NIH source  https://nihcc.app.box.com/v/ChestXray-NIHCC

## Necessary instalations

In [18]:
#!pip install plotly --upgrade --user
#!pip install pandas --upgrade --user
#!pip install numpy --upgrade --user
#!pip install scipy --upgrade --user
#!pip install scikit-learn --upgrade --user
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --user
#!pip install holoviews plotly

## Import files for metadata analysis

Get the files of metadata images so that an assesment can be made on the adequate preprocessing in order for the training to be fruitful.

In [9]:
import pandas as pd

#we import the files from our system directly downloaded from the official NIH site
bbox_data = pd.read_csv('./labels/Bbox_List_2017.csv', delimiter=',')
metadata = pd.read_csv('./labels/Data_Entry_2017_v2020.csv', delimiter=',')

In [ ]:
#we output the head of the pandas dataset for the bboxes
display(bbox_data.head())

In [ ]:
metadata.head(20)

## Small preprocessing

In [12]:
unique_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',  
                 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule',  
                 'Pleural_Thickening', 'Pneumonia','Pneumothorax']

positive_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',  
                 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
                 'Pneumonia','Pneumothorax']

core8_labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltrate', 'Mass',
                'Nodule', 'Pneumonia', 'Pneumothorax']


for label in unique_labels:
    metadata[label] = metadata['Finding Labels'].apply(lambda x: 1 if label in x else 0)

bbox_data = bbox_data.drop(columns=['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])

In [ ]:
bbox_data.rename(columns={"Bbox [x	": "x","h]": "h"})

display(bbox_data.head())

In [ ]:
display(metadata.head())

## Split the data

Taking into account the consideration of patients of the same ID can't be split within test and train in order to mantain the test as a never seen set of images for the model (images from the same patient look alike and usally share some labels)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

# Get a group shuffle split object with the desired size
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)

# Split the metadata indexes according to the group shuffle
train_idx, test_idx = next(gss.split(metadata, groups=metadata['Patient ID']))

# Create the pandas datasets using 
train_metadata = metadata.iloc[train_idx]
test_metadata = metadata.iloc[test_idx]

# Check the proportions of the resulting splits
print(f"Train set size: {len(train_metadata)}")
print(f"Test set size: {len(test_metadata)}")
print(f"Total set size: {len(train_metadata) + len(test_metadata)}")
print(f"Number of unique patients in train set: {train_metadata['Patient ID'].nunique()}")
print(f"Number of unique patients in test set: {test_metadata['Patient ID'].nunique()}")

## Statistical description of metadata

Here we can see the mean and standard deviation of all variables in order to get an understanding of the numerical properties

In [ ]:
# Selecting only the numerical columns
numerical_cols = train_metadata.select_dtypes(include=['int', 'float']).columns.tolist()

# Descriptive statistics for numerical variables
display(train_metadata[numerical_cols].describe().T)

In [ ]:
# Selecting only the categorical columns
categorical_cols = train_metadata.select_dtypes(include='object').columns.tolist()

# Descriptive statistics for categorical variables
display(train_metadata[categorical_cols].describe(include='all').T)

## Correlation analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

corr = train_metadata.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.subplots(figsize=(10, 8))
sns.heatmap(corr, mask=mask, cmap='seismic',  center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
corr2 = train_metadata[unique_labels].corr(numeric_only=True)
mask = np.triu(np.ones_like(corr2, dtype=bool))
plt.subplots(figsize=(10, 8))
sns.heatmap(corr2, mask=mask, cmap='seismic',  center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Simple visualization

Some simple graphs to understand the distribution and potential trends in data

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
data_big = ['Patient Age']
train_metadata[data_big].boxplot()

In [ ]:
plt.figure(figsize=(7, 5))
sns.violinplot(y='Patient Age', data=train_metadata, hue='Patient Gender', split=True, palette="Set2")
plt.xlabel('Gender')
plt.ylabel('Patient Age')
plt.title('Split Violin Plot of Patient Age by Gender')
plt.tight_layout()
plt.show()

In [ ]:
label_counts = train_metadata[unique_labels].sum()

label_counts = label_counts.sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette="viridis")

plt.xticks(rotation=45)
plt.xlabel('Labels')
plt.ylabel('Counts')
plt.title('Counts of Each Label')
plt.tight_layout() 

plt.show()

In [ ]:
label_counts = train_metadata[positive_labels].sum()

label_counts = label_counts.sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=label_counts.index, y=label_counts.values, palette="viridis")

plt.xticks(rotation=45)
plt.xlabel('Labels')
plt.ylabel('Counts')
plt.title('Counts of Each Label')
plt.tight_layout() 

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Group by 'Patient ID' and take the maximum for each label to count each label per patient
patient_label_counts = train_metadata.groupby('Patient ID')[positive_labels].max()

# Step 2: Sum up the number of patients with each label
label_counts_per_patient = patient_label_counts.sum()

# Step 3: Sort the counts in descending order for plotting
label_counts_per_patient = label_counts_per_patient.sort_values(ascending=False)

# Step 4: Plotting the bar plot
plt.figure(figsize=(10,6))
sns.barplot(x=label_counts_per_patient.index, y=label_counts_per_patient.values, palette="viridis")

plt.xticks(rotation=45)
plt.xlabel('Labels')
plt.ylabel('Counts (Patients)')
plt.title('Counts of Each Label Across Patients')
plt.tight_layout() 

plt.show()

In [ ]:
#Full GPT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import random

# Positive labels as provided
positive_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',
                   'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
                   'Pneumonia', 'Pneumothorax']

# Assuming 'Image Index' column has the image file names, and they are in the 'images/' folder
image_folder = './images/'

# Filter each label to get cases where only one illness is present (binary 1 for that label, 0 for others)
single_label_images = []

for label in positive_labels:
    # Filter rows where only this label is 1 and all others are 0
    filtered_df = train_metadata[(train_metadata[positive_labels].sum(axis=1) == 1) & (train_metadata[label] == 1)]
    
    # Get the image indices for these rows
    image_indices = filtered_df['Image Index'].values
    
    # If there are any images, select one randomly
    if len(image_indices) > 0:
        chosen_image = random.choice(image_indices)
        single_label_images.append(chosen_image)

# Find an image with no findings (all labels are 0)
no_findings_df = train_metadata[train_metadata[positive_labels].sum(axis=1) == 0]
no_findings_image = None
if len(no_findings_df) > 0:
    no_findings_image = random.choice(no_findings_df['Image Index'].values)

if no_findings_image:
    single_label_images.append(no_findings_image)
    positive_labels.append('No Findings')  # Add this to the labels for title display

# Create a 3x5 grid to display the images
fig, axes = plt.subplots(3, 5, figsize=(15, 9))  # 3 rows, 5 columns

# Flatten axes for easier iteration
axes = axes.flatten()

# Load and display the images in a composite (grid) of 15 images
for i, img_index in enumerate(single_label_images):
    # Load the image from the folder
    img_path = os.path.join(image_folder, img_index)
    img = Image.open(img_path)
    
    # Display the image in the grid in grayscale
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(positive_labels[i])
    axes[i].axis('off')  # Hide axes

plt.tight_layout()
plt.show()

## PCA Analysis of labels

In order to futher understand the possible clusters within the data some PCA analysis will be done to reduce dimensionality of relevant data in order to try and forsee possible trends of the model once trained.

Non-useful variables like id or image size are taken out of the PCA study. Since we are looking for clusters that are related through medical related features like the labels or patients age and gender trying to look for trends.

In [ ]:
from sklearn.preprocessing import StandardScaler

train_metadata_pre = train_metadata.copy()

# Non-useful variables like id or image size are taken out of the PCA study
train_metadata_pre = train_metadata_pre.drop(columns=['Image Index', 'Finding Labels', 'Follow-up #',
                                                      'Patient ID', 'View Position', 
                                                      'OriginalImage[Width', 'Height]',
                                                      'OriginalImagePixelSpacing[x', 'y]',])

# The variables selected are related to the medical state of the patient
study_variables =  ['Patient Age','Patient Gender','Atelectasis', 'Cardiomegaly',
                    'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 
                    'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 
                    'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

# Mild preprocessing is needed to eliminate the last categorical variable
train_metadata_pre['Patient Gender'] = train_metadata_pre['Patient Gender'].map({'M':0, 'F':1})

train_metadata_sd = train_metadata_pre.copy()

sc = StandardScaler()
train_metadata_sd[train_metadata_pre.columns] =  sc.fit_transform(train_metadata_pre)

display(train_metadata_sd.describe().T)

In [ ]:
from sklearn.decomposition import PCA
myPCA = PCA().fit(train_metadata_sd)

print(myPCA.explained_variance_ratio_)
print()
print(myPCA.explained_variance_ratio_.cumsum())

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(range(1,len(myPCA.singular_values_ )+1),myPCA.singular_values_ ,alpha=0.8,marker='.')
y_label = plt.ylabel('Eigenvalues')
x_label = plt.xlabel('Componentes')
plt.title('Scree plot')

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.plot(range(1,len(myPCA.explained_variance_ratio_ )+1),myPCA.explained_variance_ratio_ ,alpha=0.8,marker='.',label="Variancia Explicada")
y_label = plt.ylabel('Variancia explicada')
x_label = plt.xlabel('Componentes')
plt.plot(range(1,len(myPCA.explained_variance_ratio_ )+1),
         np.cumsum(myPCA.explained_variance_ratio_),
         c='red',marker='.',
         label="Variancia explicada acumulativa")
plt.legend()
plt.title('Porcentaje de variancia explicada por componente')

In [ ]:
sns.heatmap(myPCA.components_, cmap='seismic',
            xticklabels=list(train_metadata_pre.columns),
            vmin=-np.max(np.abs(myPCA.components_)),
            vmax=np.max(np.abs(myPCA.components_)),
            annot=True);

In [ ]:
principalComponents = myPCA.transform(train_metadata_sd)

loadings = myPCA.components_[:2].T * np.sqrt(myPCA.explained_variance_[:2])

plt.figure(figsize=(10, 7))

for i, feature in enumerate(train_metadata_sd.columns):
    plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.5)
    plt.text(loadings[i, 0] * 1.15, loadings[i, 1] * 1.15, feature, color='g', ha='center', va='center')

plt.xlabel(f"PC1 ({myPCA.explained_variance_ratio_[0]:.2%} variance)")
plt.ylabel(f"PC2 ({myPCA.explained_variance_ratio_[1]:.2%} variance)")
plt.title('PCA Biplot (Using Precomputed PCA)')
plt.grid()

plt.tight_layout()
plt.show()

## t-distributed Stochastic Neighbor Embedding (t-SNE) 

Since PCA assumes that variables can be linearly combined our predominantly one hot encoded dataset holds no real interest since the values are binary instead of contiuous (which usually hold greater lineal combination).

Because of that, t-SNE is a potentially better option since it is a non-linear dimensionality reduction technique since it focuses on preserving the local structure of the data .

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(train_metadata_sd)

plt.figure(figsize=(8, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5)
plt.title("t-SNE visualization")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show()

In [ ]:
# Base color mapping for individual labels
base_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema',  
               'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule',  
               'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

# Generate a palette for the base labels
palette = sns.color_palette("hsv", len(base_labels))  # Using 'hsv' for distinct but gradient-like colors

# Create a dictionary that maps each base label to a unique color
base_color_mapping = dict(zip(base_labels, palette))

# Function to blend colors for combined labels
def blend_colors(label, base_color_mapping):
    labels = label.split('|')  # Split the combined label into individual labels
    if len(labels) == 1:
        return base_color_mapping[labels[0]]  # Return the single color if it's not a combined label
    
    # Blend the colors for combined labels by averaging their RGB values
    blended_color = np.mean([base_color_mapping[l] for l in labels if l in base_color_mapping], axis=0)
    return blended_color

# Extract combined labels
combined_labels = train_metadata['Finding Labels']

label_counts = combined_labels.value_counts()

top_labels = label_counts.head(20).index


filtered_indices = combined_labels.isin(top_labels)
filtered_labels = combined_labels[filtered_indices]


colors = [blend_colors(label, base_color_mapping) for label in filtered_labels]


def plot_tsne_with_blended_colors(tsne_results, colors, top_labels, title):

    plt.figure(figsize=(12, 8))

    scatter = plt.scatter(tsne_results[filtered_indices, 0], tsne_results[filtered_indices, 1],
                          c=colors, alpha=0.7, s=10)

    handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=blend_colors(label, base_color_mapping),
                          markersize=10, label=label) for label in top_labels]
    
    plt.legend(handles=handles, title='Labels', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(title)
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.tight_layout()
    plt.show()

plot_tsne_with_blended_colors(tsne_results, colors, top_labels, "t-SNE with Blended Colors for Combined Labels")

## Bounding Box Heatmap

Let's generate a heatmap for each illness using the bbox to gain an understanding on where the pathologies are usually located from humans. This will help in the assesment of the Grad-CAM localization generated

In [ ]:
print(bbox_data['Finding Label'].unique())

In [55]:
grid_size = (1024, 1024)

heatmaps = {label: np.zeros(grid_size) for label in core8_labels}

for idx, row in bbox_data.iterrows():
    label = row['Finding Label']
    x = row['Bbox [x']
    y = row['y']
    w = row['w']
    h = row['h]']
    
    # Convert bbox to integer coordinates (assuming your bbox is float, adjust as needed)
    x, y, w, h = int(float(x)), int(float(y)), int(float(w)), int(float(h))
    
    # Increment the heatmap grid cells for the given bounding box
    if label in heatmaps:
        heatmaps[label][y:y+h, x:x+w] += 1  # Increase the count in the bbox area

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming core8_labels is defined and heatmaps dictionary contains data for each label

# Number of labels to plot (core8 suggests 8 labels)
num_labels = len(core8_labels)

# Create a 2x4 grid for the 8 heatmaps
fig, axes = plt.subplots(2, 4, figsize=(20, 12))  # 2 rows, 4 columns

# Flatten axes for easier iteration
axes = axes.flatten()

# Plot heatmaps for each label
for i, label in enumerate(core8_labels):
    sns.heatmap(heatmaps[label], cmap="hot", ax=axes[i], cbar=True)
    axes[i].set_title(f"Heatmap for {label}", fontsize=14)
    axes[i].set_xlabel("X Coordinate", fontsize=10)
    axes[i].set_ylabel("Y Coordinate", fontsize=10)

# Adjust layout to prevent overlapping of titles and labels
plt.tight_layout()
plt.show()

## Co-ocurrence matrix of the labels

In [24]:
import networkx as nx

# Step 1: Initialize an empty co-occurrence matrix
co_occurrence_matrix = pd.DataFrame(0, index=positive_labels, columns=positive_labels)

# Step 2: Compute co-occurrences between labels
for i in range(len(train_metadata)):
    # Get all labels present for the current patient (i.e., where one-hot encoding is 1)
    present_labels = train_metadata.iloc[i][positive_labels].index[train_metadata.iloc[i][positive_labels] == 1].tolist()
    
    # Update the co-occurrence matrix for every pair of present labels
    for label1 in present_labels:
        for label2 in present_labels:
            if label1 != label2:
                co_occurrence_matrix.loc[label1, label2] += 1


In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

edges = []
max_value = co_occurrence_matrix.max().max() 
for label1 in co_occurrence_matrix.index:
    for label2 in co_occurrence_matrix.columns:
        if co_occurrence_matrix.loc[label1, label2] > 0:
            edges.append((label1, label2, co_occurrence_matrix.loc[label1, label2],(co_occurrence_matrix.loc[label1, label2]/max_value)*10))

chord_data = pd.DataFrame(edges, columns=['source', 'target', 'weight', 'edge_width'])

chord = hv.Chord(chord_data)

chord.opts(
    opts.Chord(
        cmap='Category20',
        edge_cmap='Category20',
        edge_color='source',
        edge_line_width='edge_width',
        labels='index',
        node_size=15,
        edge_alpha=0.5,
        node_color='index',
        title="Co-Occurrence Chord Diagram of Positive Labels",
        height=800, width=800
    )
)

# Display the chord diagram
chord